# Projet de Machine Learning for data sciences
Utilisation des données internet sur les articles publiés sur https://news.bitcoin.com/ et les données marché pour prédire la montée ou la descente du BitCoin.  

## Import des packages
Les fonctions créées pour le projet seront expliquée lors de l'utilisation.

In [1]:
import pandas as pd #pandas
import data_collecting.api_bitcoin as bitcoin_api #data making
from importlib import reload
from datetime import datetime,timedelta
from data_processing.from_html_to_txt import extract_content_from_html as ext_html
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Il faut lancer web_knowledge.scrapper sur l'invite de commande (ou utiliser les données déjà récoltées). 
        ==> dossier wayback_dowloads
> python web_knowledge/scrapper.py 'https://news.bitcoin.com'

In [2]:
#Récupération des données de binance :
data = bitcoin_api.get_historical_data(start_date='2017-08-17') # On est limité à un interval de 1 semaine pour s'accorder aux données web

## Données disponibles grâce à l'appel API binance :
- Open time: The timestamp (in milliseconds) indicating when the candlestick period starts.
- Open: The opening price of the asset for the candlestick period.
- High: The highest price of the asset during the candlestick period.
- Low: The lowest price of the asset during the candlestick period.
- Close: The closing price of the asset for the candlestick period.
- Volume: The trading volume of the asset for the candlestick period.
- Close time: The timestamp (in milliseconds) indicating when the candlestick period ends.
- Quote asset volume: The trading volume of the quote asset for the candlestick period.
- Number of trades: The number of trades that occurred during the candlestick period.
- Taker buy base asset volume: The volume of the base asset involved in taker trades during the candlestick period.
- Taker buy quote asset volume: The volume of the quote asset involved in taker trades during the candlestick period.
- Ignore: This column is typically ignored and contains no relevant data.

In [3]:
columns=["Open time","Open","High","Low","Close","Volume","Close time","Quote asset volume","Number of trades","Taker buy base","Taker buy quote","Ignore"]
Binance_dataframe=pd.DataFrame(data,columns=columns)

On va maintenant mettre le close price du moment n+1 sur le moment n pour avoir une target dans le futur

In [4]:
Binance_dataframe['Close_n1']=0
Binance_dataframe['Close_1n']=0
for i in range(1,len(Binance_dataframe['Close'])-1):
    Binance_dataframe['Close_n1'][i]=Binance_dataframe['Close'][i+1]
    Binance_dataframe['Close_1n'][i]=Binance_dataframe['Close'][i-1]

C:\Users\lenovo\AppData\Local\Temp\ipykernel_9748\1830256754.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Binance_dataframe['Close_n1'][i]=Binance_dataframe['Close'][i+1]
C:\Users\lenovo\AppData\Local\Temp\ipykernel_9748\1830256754.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Binance_dataframe['Close_1n'][i]=Binance_dataframe['Close'][i-1]


### On va maintenant organiser nos données pour permettre la jointure avec les données web

In [5]:
open_time=[datetime.strftime(datetime.fromtimestamp(i/1000),format='%Y%m%d') for i in Binance_dataframe["Open time"]]
Binance_dataframe["Open time"]=open_time

In [6]:
#Sauvegarde des données
Binance_dataframe.to_csv("data/market/Binance_data.csv",index=False)

### On va maintenant convertir nos fichiers html en txt (formaté)

In [7]:
files = os.listdir("wayback_downloads")
log=''
for i in files:
    already_done = os.listdir("data/html/")
    if f'{i.split(".")[0][:8]}.txt' not in already_done:
        try:
            ext_html("wayback_downloads/" + i, ['div', 'p', 'h6', 'h4'], f'data/html/{i.split(".")[0][:8]}.txt')
        except Exception as e:
            log+=f"Failed to process {i}: {e}\n"
            with open("log.txt","w") as f: f.write(log)

## Création du dataframe pour ACM
On utilise nos fichiers HTML formaté en txt pour compter les mots

In [8]:
from data_processing.text_processing import proc_make_csv as make_csv

Ici on lemmatise notre texte pour qu'il ne contienne que les mots importants (pas générique) du texte.
Réutilisation d'un script fait en Analyse textuelle

In [9]:
#PARTIE SKIPPABLE, PASSER A LA SUITE POUR ECONOMISER 10 MINUTES
from data_processing.file_cleaning import delete_empty_txt_files as clean
clean(r'C:\Users\lenovo\Desktop\Cours\35_ML4DS\data\html')
word_count=make_csv()
word_count.index = [i[:8] for i in os.listdir("data/html/")]
word_count.to_csv("data/temp/word_count_og.csv")

In [10]:
word_count=pd.read_csv("data/temp/word_count_og.csv",index_col=0)

In [11]:
to_drop=[]
word_count=word_count.T
for i in range(len(word_count.index)):
    cache=word_count.iloc[i].value_counts()
    cache=pd.DataFrame(data=cache.to_numpy().flatten(),index=cache.index.to_numpy().flatten(),columns=["count"])
    if 0 not in cache.index:
        to_drop.append(word_count.index[i])
    elif int(cache.iloc[0])>len(word_count.columns)*0.5:
        to_drop.append(word_count.index[i])

In [12]:
len(word_count.index)-len(to_drop)

89

On garde alors 89 mots sur les plus de 9000 existants ==> Analyse globale permettant clustering

In [13]:
word_count.drop(to_drop,inplace=True)
word_count.to_csv("data/temp/word_count.csv") #check point

In [14]:
word_count

,20151207,20160121,20160215,20160220,20160221,20160229,20160409,20160505,20160602,20160617,...,20230506,20230508,20230509,20230510,20230511,20230512,20230513,20230514,20230515,20230516
buy,26,26,26,26,26,26,20,20,30,20,...,0,0,0,0,0,10,10,0,0,0
wallet,6,6,25,25,25,25,59,32,49,30,...,0,0,0,0,0,0,11,10,11,11
bitcoin,210,210,153,153,153,187,303,310,298,367,...,32,57,107,86,76,73,93,42,57,126
get,24,24,24,24,24,6,7,27,17,60,...,0,0,0,0,0,0,0,10,10,10
start,6,6,6,6,6,6,13,13,13,13,...,0,12,20,22,12,22,21,10,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cryptocurrencies,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11,0,0
token,0,0,0,0,0,0,0,0,0,0,...,0,10,10,10,10,22,20,0,22,0
hour,0,0,0,0,0,0,0,0,0,0,...,140,102,174,160,184,186,184,124,173,161
ago,0,0,0,0,0,0,0,0,0,0,...,438,438,438,438,438,422,422,406,390,390


On formate les données extraite des pages html pour n'avoir qu'un enregistrement par semaine pour s'accorder aux données de marché.

In [15]:
word_count=pd.read_csv("data/temp/word_count.csv",index_col=0)
word_count=word_count.T
word_count['Open time']=0
Binance_dataframe=pd.read_csv("data/market/Binance_data.csv")
word_count.index=[int(i) for i in word_count.index]
index_to_keep=[]
for i in range(len(Binance_dataframe['Open time'])):
    borne=int(datetime.strftime(datetime.strptime(str(Binance_dataframe["Open time"][i]),'%Y%m%d'),format='%Y%m%d'))
    cache=max(word_count.index[(word_count.index<=borne)])
    if cache not in index_to_keep:
        index_to_keep.append(cache)
        word_count['Open time'][cache]=Binance_dataframe['Open time'][i]

In [16]:
word_count.loc[index_to_keep].to_csv("data/temp/word_count_indexed.csv") #TO ACM

Veuillez lancer le script R dans "data_processing/clustering.r"

In [17]:
len(os.listdir("data/html/"))

945

In [18]:
len(word_count[word_count['Open time']!=0])

906

In [19]:
to_merge=pd.read_csv("data/temp/cluster_web.csv",index_col=0)
to_merge['Open time']=word_count['Open time'].values
final_data=pd.merge(Binance_dataframe,to_merge[['Open time','clust']],on='Open time')
final_data.to_csv("data/final_data.csv",index=False) #TO MODELISATION

In [20]:
final_data

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base,Taker buy quote,Ignore,Close_n1,Close_1n,clust
0,20170817,4261.48,4485.39,4200.74,4285.08,795.150377,1503014399999,3.454770e+06,3427,616.248541,2.678216e+06,0,0.00,0.00,1
1,20170823,4040.00,4265.80,4013.89,4114.01,1001.136565,1503532799999,4.148686e+06,8629,309.419092,1.293567e+06,0,4316.01,4040.00,1
2,20170830,4564.52,4647.51,4416.01,4555.14,808.468771,1504137599999,3.683715e+06,5551,196.062252,8.946202e+05,0,4724.89,4587.48,1
3,20170902,4796.16,4939.19,4286.87,4472.14,929.148595,1504396799999,4.257724e+06,7475,379.199863,1.738334e+06,0,4509.08,4834.91,1
4,20170903,4508.50,4714.76,4298.33,4509.08,691.216198,1504483199999,3.121053e+06,6022,258.934536,1.169645e+06,0,4100.11,4472.14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,20230512,26968.61,27091.12,25811.46,26795.01,67207.934940,1683935999999,1.777052e+09,1351633,33147.272510,8.764872e+08,0,26775.28,26968.62,3
902,20230513,26795.01,27045.45,26692.03,26775.28,22814.904210,1684022399999,6.120629e+08,605049,10959.299470,2.940389e+08,0,26917.62,26795.01,3
903,20230514,26775.27,27200.00,26560.53,26917.62,21594.803600,1684108799999,5.808017e+08,629537,10672.149830,2.870601e+08,0,27162.14,26775.28,3
904,20230515,26917.61,27663.59,26726.00,27162.14,40430.083330,1684195199999,1.104633e+09,935914,19301.246310,5.272376e+08,0,27033.84,26917.62,3


# On passe maintenant à la modélisation
[NOTEBOOK MODELISATION](modelisation.ipynb)